<a href="https://colab.research.google.com/github/cubecloud/fool_game/blob/master/fool_game_and_TD3_env_steps_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### New Import

In [1]:
!pip install git+https://github.com/cubecloud/fool_game.git@feature-07-td3-refactoring

  Cloning https://github.com/cubecloud/fool_game.git (to revision feature-07-td3-refactoring) to /tmp/pip-req-build-qrzb9n96
  Running command git clone -q https://github.com/cubecloud/fool_game.git /tmp/pip-req-build-qrzb9n96
  Running command git checkout -b feature-07-td3-refactoring --track origin/feature-07-td3-refactoring
  Switched to a new branch 'feature-07-td3-refactoring'
  Branch 'feature-07-td3-refactoring' set up to track remote branch 'feature-07-td3-refactoring' from 'origin'.
  Created wheel for fool-game: filename=fool_game-0.2.65-cp37-none-any.whl size=29470 sha256=d34e3dc8431d20103a3615a8eb3edb63826b74467ebb8c608458f636c96b64fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-f62e4xjt/wheels/38/c2/4e/28e56445c7bf68d7fa9b8fd53058209ed98fe03bf84a54a441
Successfully built fool-game


In [2]:
# adding terra_ai location for using googlesync 
# for development on local drive and testing 
# in google colab or jupyter notebook 
# change this variables to your locations for development
local_drive = '/home/cubecloud/GDrive'
remote_drive = '/content/drive/MyDrive'
local_dev = '/Python/fool_game/'
remote_dev = '/Python/fool_game/'
__demo_version__ = "0.1.0"
import sys
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    # from google.colab import drive
    # drive.mount('/content/drive')
    DEV = f'{remote_drive}{remote_dev}'
    DRIVE = remote_drive
elif 'ipykernel' in str(get_ipython()):
    print('Running on Jupyter Notebook')
    DEV = f'{local_drive}{local_dev}'
    DRIVE = local_drive
else:
    sys.exit('Not running on CoLab or Jupyter notebook')
print(f'Adding sys path: {DEV}')
sys.path.append(DEV)
HOME = f'{DEV}data/'

#check environment 

import tensorflow
print('Checking key environment depenndecies')
!python --version
print('TensorFlow', tensorflow.__version__)
print('Keras', tensorflow.keras.__version__)


Running on CoLab
Adding sys path: /content/drive/MyDrive/Python/fool_game/
Checking key environment depenndecies
Python 3.7.10
TensorFlow 2.5.0
Keras 2.5.0


In [3]:
import collections
import numpy as np # импортируем библиотеку для работы с массивами данных
import tensorflow as tf
# from tensorflow.keras.models import Model, load_model 
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Dense, Flatten, Input, Lambda, Conv2D, MaxPooling2D, Reshape, Multiply # из кераса загружаем необходимые слои для нейросети
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.optimizers import RMSprop, Adam, SGD, RMSprop# из кераса загружаем выбранный оптимизатор
import time                                # модуль для операций со временными характеристиками
import matplotlib.pyplot as plt            # импортируем библиотеку для визуализации данных
import seaborn as sns
import matplotlib
import pandas as pd
import pickle as pkl
import random
import sys
import os
from datetime import datetime, timedelta
from IPython.display import clear_output 

import pytz
timezone = pytz.timezone("Europe/Moscow")
# # "магическая" команда python для запуска библиотеки в ноутбуке
# %matplotlib inline

HOME = f'/content/drive/MyDrive/Python/fool_game/data/'
from google.colab import drive
drive.mount('/content/drive')

from importlib import reload
from cardgames import foolgame
foolgame = reload(foolgame)
print(tf.__version__)
print(tf.keras.__version__)
print(foolgame.__version__)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2.5.0
2.5.0
0.02.65


### Figshow

In [4]:
def figshow(df):
    fig = plt.figure(figsize=(26, 7))
    sns.set_style("white")
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.set_axisbelow(True)
    ax1.minorticks_on()
    ax1.grid(which='major', linestyle='-', linewidth='0.5', color='gray')
    ax1.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')
    last_game = int(df["game_episode"].max())
    # N = np.arange(0, last_game+1)
    n_games = 1
    if last_game // 400 > 0:
        n_games = last_game // 400
    

    plt.plot(df.loc[(df["game_episode"] % n_games == 0), "game_episode"], 
             df.loc[(df["game_episode"] % n_games == 0), "loss"], 
             linestyle='--', color='blue', label="loss")
    plt.plot(df.loc[(df["game_episode"] % n_games == 0), "game_episode"], 
             df.loc[(df["game_episode"] % n_games == 0), "epsilon"], 
             linestyle='--', color='green', label="epsilon")
    plt.title(f"Loss & epsilon")
    plt.legend()

    ax2 = fig.add_subplot(1, 2, 2)
    ax2.set_axisbelow(True)
    ax2.minorticks_on()
    ax2.grid(which='major', linestyle='-', linewidth='0.5', color='gray')
    ax2.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')
    plt.plot(df.loc[(df["game_episode"] % n_games == 0), "game_episode"],
             df.loc[(df["game_episode"] % n_games == 0), 'mean_reward'], 
             linestyle='-', color='red', label="mean_reward")
    plt.title(f"mean_reward")
    plt.legend()
    plt.show()
    pass

In [5]:
!nvidia-smi

Wed May 26 07:13:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## The DQN model


In [6]:
import torch
import torch.nn as nn        # Pytorch neural network package
import torch.optim as optim  # Pytorch optimization package
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn.functional as F
from tensorboardX import SummaryWriter

# device = torch.device("cuda")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Initialize parameters and build model.
        Args:
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            max_action (float): highest action to take
            seed (int): Random seed
            h1_units (int): Number of nodes in first hidden layer
            h2_units (int): Number of nodes in second hidden layer
            
        Return:
            action output of network with tanh activation
    """
    
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, action_dim)

        self.max_action = max_action


    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.max_action * torch.tanh(self.l3(x)) 
        return x

In [8]:
class Critic(nn.Module):
    """Initialize parameters and build model.
        Args:
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            max_action (float): highest action to take
            seed (int): Random seed
            h1_units (int): Number of nodes in first hidden layer
            h2_units (int): Number of nodes in second hidden layer
            
        Return:
            value output of network 
    """
    
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(state_dim + action_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, 1)

        # Q2 architecture
        self.l4 = nn.Linear(state_dim + action_dim, 400)
        self.l5 = nn.Linear(400, 300)
        self.l6 = nn.Linear(300, 1)


    def forward(self, x, u):
        xu = torch.cat([x, u], 1)

        x1 = F.relu(self.l1(xu))
        x1 = F.relu(self.l2(x1))
        x1 = self.l3(x1)

        x2 = F.relu(self.l4(xu))
        x2 = F.relu(self.l5(x2))
        x2 = self.l6(x2)
        return x1, x2


    def Q1(self, x, u):
        xu = torch.cat([x, u], 1)

        x1 = F.relu(self.l1(xu))
        x1 = F.relu(self.l2(x1))
        x1 = self.l3(x1)
        return x1

In [9]:
# import warnings
# warnings.filterwarnings('ignore')

In [10]:


# Code based on: 
# https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py

# Expects tuples of (state, next_state, action, reward, done)
class ReplayBuffer(object):
    """Buffer to store tuples of experience replay"""
    
    def __init__(self, max_size=1000000):
        """
        Args:
            max_size (int): total amount of tuples to store
        """
        
        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def add(self, data):
        """Add experience tuples to buffer
        
        Args:
            data (tuple): experience replay tuple
        """
        
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(data)

    def sample(self, batch_size):
        """Samples a random amount of experiences from buffer of batch size
        
        Args:
            batch_size (int): size of sample
        """
        
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        states, actions, next_states, rewards, dones = [], [], [], [], []

        for i in ind: 
            s, a, s_, r, d = self.storage[i]
            states.append(np.array(s, copy=False))
            actions.append(np.array(a, copy=False))
            next_states.append(np.array(s_, copy=False))
            rewards.append(np.array(r, copy=False))
            dones.append(np.array(d, copy=False))

        return np.array(states), np.array(actions), np.array(next_states), np.array(rewards).reshape(-1, 1), np.array(dones).reshape(-1, 1)



#### old experience

In [11]:
# Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'valid_action_lst','reward', 'done', 'next_state'])

# class ExperienceReplay:
#     def __init__(self, capacity):
#         self.capacity = capacity
#         self.buffer = collections.deque(maxlen=capacity)
#         pass

#     def __len__(self):
#         return len(self.buffer)

#     def get_length(self):
#         return self.__len__()

#     def append(self, experience):
#         self.buffer.append(experience)
#         pass

#     def sample(self, batch_size):
#         indices = np.random.choice(len(self.buffer), batch_size, replace=False)
#         states, actions, valid_actions_lst, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
#         return (np.array(states, dtype=np.float32), 
#                 np.array(actions, dtype=np.int64), 
#                 np.array(valid_actions_lst), 
#                 np.array(rewards, dtype=np.float32).reshape(-1, 1),
#                 np.array(dones, dtype=np.uint8).reshape(-1, 1), 
#                 np.array(next_states, dtype=np.float32))

#     def save(self, file_path, buffer_length=10000):
#         len_buffer = len(self.buffer)
#         with open(file_path, "wb") as f:
#             print('Save exp buffer...')
#             if not (self.capacity is None) \
#                     and (len_buffer < self.capacity) \
#                     and (len_buffer < buffer_length):
#                 buffer_length = len_buffer
#             else:
#                 buffer_length = len_buffer
#             states, actions, valid_actions_lst, rewards, dones, next_states = \
#                 zip(*[self.buffer[idx] for idx in range(len(self.buffer) - buffer_length, len(self.buffer))])
#             pkl.dump([states, actions, valid_actions_lst, rewards, dones, next_states], f)
#             del [states, actions, valid_actions_lst, rewards, dones, next_states]
#             pass

#     def load(self, file_path):
#         with open(file_path, "rb") as f:
#             print('Loading exp buffer...')
#             # self.buffer = pkl.load(f)
#             states, actions, valid_actions_lst, rewards, dones, next_states = pkl.load(f)
#             for state, action, valid_action_lst, reward, done, next_state in zip(states, actions, valid_actions_lst, rewards, dones, next_states):
#                 exp = Experience(state, action, valid_action_lst, reward, done, next_state)
#                 self.buffer.append(exp)
#             del [states, actions, valid_actions_lst, rewards, dones, next_states]
#         pass

### TD3 Agent

In [12]:
class TD3(object):
    """Agent class that handles the training of the networks and provides outputs as actions
    
        Args:
            state_dim (int): state size
            action_dim (int): action size
            max_action (float): highest action to take
            device (device): cuda or cpu to process tensors
            env (env): gym environment to use
    
    """
    
    def __init__(self, state_dim, action_dim, max_action, env):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=1e-3)

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=1e-3)

        self.max_action = max_action
        self.env = env

        
    def select_action(self, state, noise=0.1):
        """Select an appropriate action from the agent policy
        
            Args:
                state (array): current state of environment
                noise (float): how much noise to add to acitons
                
            Returns:
                action (float): action clipped within action range
        
        """
        
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        
        action = self.actor(state).cpu().data.numpy().flatten()
        # print('action from actor',action)
        if noise != 0: 
            action = (action + np.random.normal(0, noise, size=action.shape))
            # action = (action + np.random.normal(0, noise, size=self.env.action_space.shape[0]))
        # print('action before clip', action)
        # action = action.clip(self.env.action_space.low, self.env.action_space.high)
        action = action.clip(-1.0, 1.0)
        # print('action after clip', action)
        return action

    
    def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
        """Train and update actor and critic networks
        
            Args:
                replay_buffer (ReplayBuffer): buffer for experience replay
                iterations (int): how many times to run training
                batch_size(int): batch size to sample from replay buffer
                discount (float): discount factor
                tau (float): soft update for main networks to target networks
                
            Return:
                actor_loss (float): loss from actor network
                critic_loss (float): loss from critic network
        
        """
        
        for it in range(iterations):

            # Sample replay buffer 
            x, y, u, r, d = replay_buffer.sample(batch_size)

            state = torch.FloatTensor(x).to(device)
            action = torch.FloatTensor(u).to(device)
            next_state = torch.FloatTensor(y).to(device)
            done = torch.FloatTensor(1 - d).to(device)
            reward = torch.FloatTensor(r).to(device)

            # Select action according to policy and add clipped noise 
            noise = torch.FloatTensor(u).data.normal_(0, policy_noise).to(device)
            noise = noise.clamp(-noise_clip, noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)

            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + (done * discount * target_Q).detach()

            # Get current Q estimates
            current_Q1, current_Q2 = self.critic(state, action)

            # Compute critic loss
            critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q) 

            # Optimize the critic
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

            # Delayed policy updates
            if it % policy_freq == 0:

                # Compute actor loss
                actor_loss = -self.critic.Q1(state, self.actor(state)).mean()

                # Optimize the actor 
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()

                # Update the frozen target models
                for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

                for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)


    def save(self, filename, directory):
        torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
        torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))


    def load(self, filename="best_avg", directory="./saves"):
        self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
        self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

In [13]:
class Runner():
    """Carries out the environment steps and adds experiences to memory"""
    
    def __init__(self, env, agent, replay_buffer):
        
        self.env = env
        self.agent = agent
        self.replay_buffer = replay_buffer
        self.obs = self.env.reset()
        self.done = False
        self._reset()
        pass

    def _reset(self):
        self.obs = self.env.reset()
        # self.env.step(0, first_step=True)
        pass

    def next_step(self, episode_timesteps, noise=0.1):
        action = self.agent.select_action(np.array(self.obs), noise=0.1)

        if not self.env.action_space.is_this_valid_action_ohe(action):
            reward = -0.007
            done_bool = False
            replay_buffer.add((self.obs, self.obs, action, reward, done_bool))
            action = self.env.action_space.sample_ohe()
            # print(action)
        
        # Perform action
        new_obs, reward, done, info = self.env.step(action) 
        done_bool = 0 if episode_timesteps + 1 == 200 else float(done)
        # print(info)
        # Store data in replay buffer
        replay_buffer.add((self.obs, new_obs, action, reward, done_bool))
        
        self.obs = new_obs
        
        if done:
            # self.obs = self.env.reset()
            done = False
            # print('Done')
            self._reset()
            return reward, True
        
        return reward, done

In [14]:
def evaluate_policy(policy, env, eval_episodes=100,render=False):
    """run several episodes using the best agent policy
        
        Args:
            policy (agent): agent to evaluate
            env (env): gym environment
            eval_episodes (int): how many test episodes to run
            render (bool): show training
        
        Returns:
            avg_reward (float): average reward over the number of evaluations
    
    """
    
    avg_reward = 0.
    for i in range(eval_episodes):
        obs = env.reset()
        done = False
        while not done:
            if render:
                env.render()
            action = policy.select_action(np.array(obs), noise=0)
            obs, reward, done, _ = env.step(action)
            avg_reward += reward

    avg_reward /= eval_episodes

    print("\n---------------------------------------")
    print("Evaluation over {:d} episodes: {:f}" .format(eval_episodes, avg_reward))
    print("---------------------------------------")
    return avg_reward

In [15]:
def observe(env,replay_buffer, observation_steps):
    """run episodes while taking random actions and filling replay_buffer
    
        Args:
            env (env): gym environment
            replay_buffer(ReplayBuffer): buffer to store experience replay
            observation_steps (int): how many steps to observe for
    
    """
    
    time_steps = 0
    obs = env.reset()
    # env.step(0, first_step=True)
    # obs = env.reset()
    done = False

    while time_steps < observation_steps:
        action = env.action_space.sample_ohe()
        if not env.action_space.is_this_valid_action_ohe(action):
            reward = -0.007
            done_bool = False
            replay_buffer.add((obs, obs, action, reward, done_bool))
            action = env.action_space.sample_ohe()
        # print('Valid actions\n', env.action_space.pl.analyze())
        # print('Action from sample\n', action)
        # print('Desktop\n', env.action_space.pl.desktop_list)
        # print('Cards on hand\n', env.action_space.pl.player_cards_onhand_list)
        new_obs, reward, done, info = env.step(action)
        # print(info)
        replay_buffer.add((obs, new_obs, action, reward, done))
        # assert new_obs.shape == obs.shape, f'Error in states dimensions {new_obs.shape} != {obs.shape}'
        obs = new_obs
        time_steps += 1

        if done:
            obs = env.reset()
            # env.step(0, first_step=True)
            done = False

        print("\rPopulating Buffer {}/{}.".format(time_steps, observation_steps), end="")
        sys.stdout.flush()

In [16]:
def train(agent, test_env):
    """Train the agent for exploration steps
    
        Args:
            agent (Agent): agent to use
            env (environment): gym environment
            writer (SummaryWriter): tensorboard writer
            exploration (int): how many training steps to run
    
    """

    total_timesteps = 0
    timesteps_since_eval = 0
    episode_num = 0
    episode_reward = 0
    episode_timesteps = 0
    done = False 
    obs = env.reset()
    evaluations = []
    rewards = []
    best_avg = -2000
    
    writer = SummaryWriter(comment="-TD3_foolgame_state_v006")
    
    while total_timesteps < EXPLORATION:
    
        if done: 

            if total_timesteps != 0: 
                rewards.append(episode_reward)
                avg_reward = np.mean(rewards[-100:])
                
                writer.add_scalar("avg_reward", avg_reward, total_timesteps)
                writer.add_scalar("reward_step", reward, total_timesteps)
                writer.add_scalar("episode_reward", episode_reward, total_timesteps)
                
                if best_avg < avg_reward and len(rewards)>100:
                    best_avg = avg_reward
                    print(f"saving best model... Best average = {best_avg:.4f} at {datetime.now(timezone)}\n")
                    agent.save("best_avg", HOME)
                    # agent.save("best_avg","saves")

                print("\rTotal T: {:d} Episode Num: {:d} Reward: {:f} Avg Reward: {:f} ".format(
                    total_timesteps, episode_num, episode_reward, avg_reward), end="")
                sys.stdout.flush()


                if avg_reward >= REWARD_THRESH:
                    break

                agent.train(replay_buffer, episode_timesteps, BATCH_SIZE, GAMMA, TAU, NOISE, NOISE_CLIP, POLICY_FREQUENCY)

                # Evaluate episode
#                 if timesteps_since_eval >= EVAL_FREQUENCY:
#                     timesteps_since_eval %= EVAL_FREQUENCY
#                     eval_reward = evaluate_policy(agent, test_env)
#                     evaluations.append(avg_reward)
#                     writer.add_scalar("eval_reward", eval_reward, total_timesteps)

#                     if best_avg < eval_reward:
#                         best_avg = eval_reward
#                         print("saving best model....\n")
#                         agent.save("best_avg","saves")

                episode_reward = 0
                episode_timesteps = 0
                episode_num += 1 

        reward, done = runner.next_step(episode_timesteps)
        episode_reward += reward

        episode_timesteps += 1
        total_timesteps += 1
        timesteps_since_eval += 1


In [17]:
# ENV = "RoboschoolHalfCheetah-v1"#"Pendulum-v0"
SEED = 0
OBSERVATION = 15000
EXPLORATION = 5000000
BATCH_SIZE = 64
GAMMA = 0.99
TAU = 0.0005
NOISE = 0.2
NOISE_CLIP = 0.5
EXPLORE_NOISE = 0.1
POLICY_FREQUENCY = 2
EVAL_FREQUENCY = 5000
REWARD_THRESH = 8000

In [18]:
# env = gym.make(ENV)
players_num = 2
env = foolgame.Environment(players_num,
                           env_type="Dummy",
                           observer_player=1,
                           nnmodel=None)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds
# env.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

# state_dim = env.observation_space.shape[0]
state_dim = env.observation_space.states_dim
action_dim = env.action_space.shape[0] 
# max_action = float(env.action_space.high[0])
''' max action probability from neurons '''
max_action = env.action_space.abs_minmax


policy = TD3(state_dim, action_dim, max_action, env)

replay_buffer = ReplayBuffer()

runner = Runner(env, policy, replay_buffer)

total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True

In [19]:
# Populate replay buffer
observe(env, replay_buffer, OBSERVATION)

Populating Buffer 15000/15000.

In [ ]:
train(policy, env)

Total T: 4365 Episode Num: 99 Reward: -0.878000 Avg Reward: 0.083330 saving best model... Best average = 0.0636 at 2021-05-26 10:14:44.202922+03:00

Total T: 4411 Episode Num: 100 Reward: -0.875000 Avg Reward: 0.063560 saving best model... Best average = 0.0636 at 2021-05-26 10:14:44.508794+03:00

Total T: 4546 Episode Num: 103 Reward: -0.861000 Avg Reward: 0.043280 saving best model... Best average = 0.0639 at 2021-05-26 10:14:45.309784+03:00

Total T: 5033 Episode Num: 113 Reward: 1.124000 Avg Reward: 0.045110 saving best model... Best average = 0.0662 at 2021-05-26 10:14:47.982529+03:00

Total T: 5083 Episode Num: 114 Reward: 1.173000 Avg Reward: 0.066220 saving best model... Best average = 0.0867 at 2021-05-26 10:14:48.336056+03:00

Total T: 5170 Episode Num: 116 Reward: -0.884000 Avg Reward: 0.066710 saving best model... Best average = 0.0870 at 2021-05-26 10:14:48.876616+03:00

Total T: 5217 Episode Num: 117 Reward: 1.160000 Avg Reward: 0.086950 saving best model... Best average 

In [ ]:
!tensorboard --logdir= runs

# Pytorch *Agent*

In [ ]:
players_num = 2
num_actions=37

# sync_target_turns = 3000       
replay_start_size = 10000       # start replay at turn_idx

model = DQN((518,), 37).to(device)
target_model = DQN((518,), 37).to(device)

# print(next(model1.parameters()).device)
# print(next(model2.parameters()).device)
# net = DQN((4536), 37).to(device)
# target_net = DQN((4536), 37).to(device)

In [ ]:
class Agent:
    def __init__(self, 
                 env, 
                 exp_buffer,
                 exp_buffer_size = 50000):
        self.env = env
        self.observer_player = env.observer_player
        self.exp_buffer = exp_buffer
        self.exp_buffer_size = exp_buffer_size
        self.verbose = False
        self.debug_verbose = 1
        self.gamma = 0.99
        self._reset()
        pass

    def _reset(self):
        self.state = np.reshape(self.env.reset(), 518)
        self.total_reward: float = 0.0
        self.env.verbose = self.verbose
        self.env.debug_verbose = self.debug_verbose
        self.env.step(0, first_step=True)
        self.turns_counter = 0
        pass

    def play_step(self, net, epsilon=0.0, step_device="cpu"):
        done_reward = None
        self.turns_counter += 1
        self.env.epsilon = epsilon
        step_valid_actions = self.env.pl[self.observer_player].analyze()
        if np.random.random() < epsilon:
            action = random.choice(step_valid_actions)
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(step_device)
            q_vals_v = net.forward(state_v)
            # action = np.argmax(q_vals_v.cpu().detach().numpy())
            # print(next(net.parameters()).device)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

            # state_a = np.array([self.state], copy=False)
            # state = torch.FloatTensor(state_a).float().unsqueeze(0).to(device)
            # qvals = net.forward(state)
            # action = np.argmax(qvals.cpu().detach().numpy())

            # state_a = np.array([self.state], copy=False)
            # state_tensor = tf.convert_to_tensor(state_a)
            # q_values = nnmodel(state_tensor, training=False)
            # # with np.printoptions(precision=3, suppress=True):
            # #     print(q_values.numpy())
            # valid_masks = tf.one_hot(valid_action_list, self.env.num_actions)
            # # with np.printoptions(precision=3, suppress=True):
            # #     print(masks.numpy())
            # valid_q_values = tf.expand_dims(tf.reduce_sum(tf.multiply(q_values, valid_masks), axis=0), 0)
            # # with np.printoptions(precision=3, suppress=True):
            # #     print(valid_q_values.numpy())
            # action = np.argmax(valid_q_values)
            # # print(self.action, action_list, action)
        if not (action in step_valid_actions):
            # action = step_valid_actions[0]
            # action = random.choice(step_valid_actions)
            ''' keeping the same state for new_state but adding negative reward'''
            new_state = np.squeeze(state_a, axis=0)
            self.state = np.squeeze(state_a, axis=0)
            # print(new_state.shape)
            # print(self.state.shape)
            ''' negative reward for not valid action '''
            reward = -0.007
            is_done = False
            self.total_reward += reward
            exp = Experience(self.state, action, step_valid_actions, reward, is_done, new_state)
            self.exp_buffer.append(exp)
            action = random.choice(step_valid_actions)
        ''' 
        step_epsilon - setting to 1.0 - totally random action from agent 
        trying to learn - how to play valid actions (index + zero (pass)) 
        '''
        new_state, reward, is_done, info = self.env.step(action, step_epsilon=1.0)
        new_state = np.reshape(new_state, 518)
        # if self.verbose:
            # print("Last action", action)
            # print(info)
        self.total_reward += reward
        exp = Experience(self.state, action, step_valid_actions, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done or self.turns_counter > 100:
            done_reward  = self.total_reward
            self.verbose = False
            self.debug_verbose = 1
            self._reset()
        return done_reward

In [ ]:
MEAN_REWARD_BOUND = 0.85         

buffer = ExperienceReplay(replay_start_size)
environment = foolgame.Environment(players_num,
                                   env_type="dummy",
                                   observer_player=1,
                                   nnmodel=None)
game_table = Agent(environment,
                   exp_buffer=buffer)

gamma = 0.99
tau = 0.00005                                   
batch_size = 128                   

eps_start = 1.0
eps_decay = .999985             #changed start from .999985
eps_min = 0.07                  

In [ ]:
model_weights_save_name = 'fool_cardgame_weights'
history_csv_name = 'fool_cardgame_hst.csv'
exp_pkl_name = 'fool_cardgame_exp.pkl'
cols = ['game_episode',
        'turns',  
        'time', 
        'loss',
        'epsilon',
        'mean_reward', 
       ]

def get_pd_data(pdfile):
  df_data = pd.read_csv(os.path.join(HOME, pdfile), index_col=0)
  return df_data

def add_data_to_pd(df_data, _game_episode, _turns, _time, _loss, _epsilon, _mean_reward):
  df_data = df_data.append({'game_episode': _game_episode,
                            'turns': _turns,
                            'time': _time,
                            'loss': _loss,
                            'epsilon': _epsilon,
                            'mean_reward': _mean_reward,
                            }, ignore_index=True)
  # print(self.df.to_string())
  return df_data
  
def load_checkpoint(save_name):
    pd_file_path =  os.path.join(HOME, history_csv_name)
    if os.path.exists(pd_file_path):    
      df = get_pd_data(pd_file_path)
      pd_start_game= int(df.game_episode.max())
    else:
      pd_start_game = 0
      df = pd.DataFrame(columns=cols)
    start_game = 0
    mean_reward = 0
    rewards_in_row = []
    dirlist = os.listdir(HOME)
    for i in range (len(dirlist)):
      filename = dirlist[i]
      if save_name in filename:
        if '_' in filename: 
          try: 
            epoch = int((filename.split('_')[-1]).split('.')[-2])
          except ValueError:
            epoch = 0
          if epoch > start_game:
            start_game = epoch
    if pd_start_game > start_game:
      df.drop(index = [i for i in range(start_game+1, pd_start_game+1)], axis=0, inplace=True)
    if start_game > 0:
      file_path = os.path.join(HOME, f'{save_name}_{start_game}.h5')
      model.load_state_dict(torch.load(file_path))
      target_model.load_state_dict(torch.load(file_path))

      print(f"Loaded {file_path} save, for model & model_target. Starting training from {start_game}")
      # last epoch +1
      epsilon = float(df.loc[(df["game_episode"]==start_game), "epsilon"].item())
      mean_reward = float(df.loc[(df["game_episode"]==start_game), "mean_reward"].item())
    else:
      epsilon = 1.0
      print(f"Starting from scratches. Starting game = {start_game}")
      rewards_in_row = []
    return df, start_game, epsilon, mean_reward, rewards_in_row

df, start_game, epsilon, mean_reward, rewards_in_row = load_checkpoint(model_weights_save_name)

if start_game != 0:
  turn_idx = int(df.turns.max())
  figshow(df)
  game_episode = start_game
  total_rewards = []
  total_rewards = df.loc[(df["game_episode"]<=start_game) &(df["game_episode"]>=start_game-100), "mean_reward"].values.tolist()
  # epsilon = 0.1
  learning_rate = 0.00025
  adam_hat = 1e-8
  game_time = 0
  if game_episode > 200:
    lr_decay = 200/game_episode
    learning_rate = learning_rate * lr_decay
    if learning_rate < adam_hat:
      learning_rate = adam_hat
else:
  turn_idx = 0
  mean_reward = 0
  game_episode = -1
  epsilon = eps_start
  total_rewards = []
  learning_rate = 0.00025 

print(df.tail(15).to_string())
print(f'Start game episode:', start_game)
print('Turn index:', turn_idx)
print('Epsilon:', epsilon)
print('Learning rate:', learning_rate)
print('Mean reward on last game episode:', mean_reward)
print(f'_mean_ of Total rewards for last 100 games: {np.mean(total_rewards[-100:]) if total_rewards else None}')

exp_file_path = os.path.join(HOME, f'{exp_pkl_name}')
if start_game != 0:
  if os.path.exists(exp_file_path):
    buffer.load(exp_file_path)
  else:
    msg = f"Warning: pickle Exp file not found, starting without" 
    print(msg)

time_sum = int(df["time"].sum())
print(f'Model learning {start_game} games and {timedelta(seconds=time_sum)}')

In [ ]:
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters())

msg_display = True
best_mean_reward = None
num_actions = 37
loss_numpy = 0
loss = 0
start_datetime = datetime.now(timezone)
print(">>>Training starts at ", start_datetime)
start_time = time.time()
episodes_start_time = time.time()

while True:
        turn_idx += 1
        if len(buffer) < replay_start_size:
            epsilon = 1.0
        reward = game_table.play_step(model, epsilon=epsilon, step_device=device)
        if reward is not None:
            epsilon = max(epsilon*eps_decay, eps_min)
            reward = np.float16(reward)
            game_episode += 1
            total_rewards.append(reward)
            if len(total_rewards) > 100:
              mean_reward = np.mean(total_rewards[-100:])
              del total_rewards[:1]
            msg = f"\rgame_episode: {game_episode:07d}, turn: {turn_idx:09d}, epsilon: {epsilon:.3f}, " \
                  f"mean reward: {mean_reward:.3f}, loss: {loss_numpy:.4f}, " \
                  f"last reward {reward}"

            if (best_mean_reward is None or best_mean_reward < mean_reward) \
              and game_episode > 1000:
                best_mean_reward = mean_reward
                if best_mean_reward is not None:
                    torch.save(model.state_dict(), os.path.join(HOME, f'{model_weights_save_name}_{game_episode:002d}.h5'))
                    msg = f"\rgame_episode: {game_episode:07d}, turn: {turn_idx:09d}, epsilon: {epsilon:.3f}, " \
                    f"mean reward: {mean_reward:.3f}, loss: {loss_numpy:.4f}, " \
                    f"last reward {reward}"
                    msg = f'{msg}. Best mean reward updated {best_mean_reward:.3f}'
                    msg_display = True

            if mean_reward > MEAN_REWARD_BOUND \
              and len(buffer) > replay_start_size \
              and epsilon == 0.02:
                end_time = time.time()
                game_time = end_time - start_time 
                start_time  = time.time()
                print(f'{msg}, time: {game_time:.4f} sec')    
                print("Solved in %d turns!" % turn_idx)
                loss_numpy = loss_t.item()
                # if tf.is_tensor(loss):
                    # loss_numpy = tf.keras.backend.get_value(loss)
                df = add_data_to_pd(df, game_episode, turn_idx, game_time, loss_numpy, epsilon, mean_reward)
                df.to_csv(os.path.join(HOME, history_csv_name), encoding='utf-8')
                break
            end_time = time.time()
            game_time = end_time - start_time 
            start_time  = time.time()

            # if tf.is_tensor(loss):
                # loss_numpy = tf.keras.backend.get_value(loss)    
            if game_episode % 100 == 0 or msg_display:
              episodes_end_time = time.time()
              episodes_time = episodes_end_time - episodes_start_time 
              episodes_start_time  = time.time() 
              print(f'{msg}, episodes time: {episodes_time:.4f} sec')
              msg_display = False
            df = add_data_to_pd(df, game_episode, turn_idx, game_time, loss_numpy, epsilon, mean_reward)
            if (game_episode % 500 == 0) and (game_episode != 0):
                df.to_csv(os.path.join(HOME, history_csv_name), encoding='utf-8')
            if (game_episode % 2500 == 0) and (game_episode != 0) and (turn_idx >= replay_start_size):
                print('Saving weights...')
                torch.save(model.state_dict(), os.path.join(HOME, f'{model_weights_save_name}_{game_episode:002d}.h5'))
                buffer.save(exp_file_path, buffer_length=50000)
            if game_episode % 1000 == 0:
                figshow(df)
            if (game_episode % 10000 == 0) and (game_episode != 0):  
                clear_output()
                print(df.tail(50).to_string())
                figshow(df)
        if len(buffer) < replay_start_size:
            continue

        batch = buffer.sample(batch_size)
        states, actions, valid_actions_lst, rewards, dones, next_states = batch

        states = torch.FloatTensor(states).to(device)
        actions = torch.LongTensor(actions).to(device)
        rewards = torch.FloatTensor(rewards).to(device)
        next_states = torch.FloatTensor(next_states).to(device)
        dones = torch.ByteTensor(dones).to(device)

        # resize tensors
        actions = actions.view(actions.size(0), 1)
        dones = dones.view(dones.size(0), 1)

        # compute loss
        curr_Q = model.forward(states).gather(1, actions)
        next_Q = target_model.forward(next_states)
        max_next_Q = torch.max(next_Q, 1)[0]
        max_next_Q = max_next_Q.view(max_next_Q.size(0), 1)
        # max_next_Q = max_next_Q.to(device)
        expected_Q = rewards + (1 - dones) * gamma * max_next_Q
        # expected_Q = expected_Q.to(device)

        loss = F.mse_loss(curr_Q, expected_Q.detach())
        # loss = nn.MSELoss()(curr_Q, expected_Q.detach)

        # states_v = torch.tensor(states).to(device)
        # next_states_v = torch.tensor(next_states).to(device)
        # actions_v = torch.tensor(actions).to(device)
        # rewards_v = torch.tensor(rewards).to(device)
        # done_mask = torch.ByteTensor(dones).to(device)

        # state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)

        # next_state_values = target_net(next_states_v).max(1)[0]

        # next_state_values[done_mask] = -1.0

        # next_state_values = next_state_values.detach()

        # expected_state_action_values = next_state_values * gamma + rewards_v

        # loss_t = nn.MSELoss()(state_action_values, expected_state_action_values)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_numpy = loss.item()
        
        # target network update
        if turn_idx % 2 == 0:
          for target_param, param in zip(target_model.parameters(), model.parameters()):
              target_param.data.copy_(tau * param + (1 - tau) * target_param)

        # if turn_idx % sync_target_turns == 0:
            # update the the target network with new weights
            # print("Synchronizing models...")
            # target_net.load_state_dict(net.state_dict())
            # model_target.set_weights(model.get_weights())
        # for target_param, param in zip(target_net.parameters(), net.parameters()):
        #     target_param.data.copy_(tau * param + (1 - tau) * target_param)

print('Saving weights...')
torch.save(model1.state_dict(), os.path.join(HOME, f'{model_weights_save_name}_{game_episode:002d}.h5'))
buffer.save(exp_file_path)
figshow(df)

end_datetime = datetime.now(timezone)
print(">>>Training ends at ", end_datetime)
train_time = end_datetime-start_datetime
print(f'Model trained:', train_time)
